In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import catboost
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool, cv
from pathlib import Path
import src.utils as u
import src.cb_utils as m
from tqdm.auto import tqdm
from collections import Counter
from sklearn.metrics import f1_score, precision_score, recall_score

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)

u.random_seed(42)

data_path = Path('../data')

In [ ]:
train_full, test, all_cols, cont_cols, cat_cols = u.read_data(data_path)

In [ ]:
# # corr = train_full[all_cols].corr()

# high_corr = []
# fields = list(set(corr.columns) - set(high_corr_to_drop))

# for f1 in fields:
#     for f2 in fields:
#         if (f1 != f2) and (corr[f1][f2] > 0.90) and ({f1, f2} not in high_corr):
#             print(f1, f2, corr[f1][f2])
#             high_corr.append({f1, f2})
            
# print(high_corr)

In [ ]:
high_corr_to_drop = ['longitude', 'latitude', 'LANES', 'lane_width', 'P0', 'dist_to_center', 'traffic1',
                    'avg_speed_neigh', 'traffic_total_neigh',
                    'average_ttime_neigh', 'acc_cnt_last_quarter_sid_neigh',
                    'mean_traffic', 'acc_cnt_last_halfyear_sid_neigh',
                    # with low feature importance
                    
                     'cloud_cover',
                     'cloud1_cover',
                     'acc_cnt_last_halfyear_sid_wd_h_neigh',
                     'delta_rel_diff_traffic_last_hour',
                     'rel_diff_traffic',
                     'acc_cnt_last_halfyear_vds',
                     'delta_traffic_total_next_hour',
                     'temp',
                     'acc_cnt_last_halfyear_vds_wd',
                     'orientation',
                     'cloud_3',
                     'wind_speed',
                     'wind_dir_angle',
                     'max_gust',
                     'weather_cond',
                     'blinding',
                     'acc_cnt_last_quarter_vds_wd',
                     'cloud_height',
                     'delta_traffic_total_last_hour',
                     'wind_dir',
                     'delta_rel_diff_traffic_next_hour',
                     'acc_cnt_last_quarter_sid_wd_h_neigh',
                     'precip_time',
                     'humidity',
                     'angle_wind_road',
                     'cloud_1',
                     'mean_avg_speed',
                     'acc_cnt_last_halfyear_vds_wd_h',
                     'cloud_2',
                     'sinuosity',
                     'wind_dir_defined',
                     'vds_count',
                     'snow',
                     'CLASS',
                     'mist',
                     'vms_count',
                     'PAVETYPE',
                     'delta_average_ttime_last_day',
                     'average_ttime_na',
                     'precip_mm',
                     'SURFTYPE',
                     'smoke',
                     'fog',
                     'drizzle',
                     'cloud_cover_fog',
                     'delta_average_ttime_next_day',
                     'WIDTH',
                     'visibility',
                     'rain',
                     'main_route']

all_cols = list(set(all_cols) - set(high_corr_to_drop))
cont_cols = list(set(cont_cols) - set(high_corr_to_drop))
cat_cols = list(set(cat_cols) - set(high_corr_to_drop))

# Cross val

In [ ]:
# train = pd.concat([u.select_by_time(train_full, '2016-01-01', '2018-01-01', 'datetime'),
#                    u.select_by_time(train_full, '2018-04-01', '2019-01-01', 'datetime')])
# train.reset_index(inplace=True)

# val = u.select_by_time(train_full, '2018-01-01', '2018-04-01', 'datetime')


# def get_folds(df):
#     months = [('2018-04-01', '2018-06-30'),
#               ('2018-07-01', '2018-09-30'),
#               ('2018-10-01', '2019-01-01')]
    
#     folds = []
#     for (ta, tb) in months:
#         tstart = pd.Timestamp(ta)
#         tend = pd.Timestamp(tb)

#         tt = df.datetime
#         ids_val = ((tstart <=  tt) & (tt < tend)).to_numpy().nonzero()[0]
#         ids_val = np.array(ids_val)
#         folds.append(ids_val)
    
#     return folds
        
    
# folds = get_folds(train)

In [ ]:
# val_pool = Pool(data=val[all_cols], label=val['y'], cat_features=cat_cols)
# test_pool = Pool(data=test[all_cols], label=test['y'], cat_features=cat_cols)

In [ ]:
# probas, ths, val_scores = m.fit_predict_cv(train=train,
#                                            test=val,
#                                            val_ids=folds, 
#                                            cat_cols=cat_cols,
#                                            all_cols=all_cols,
#                                            draw_plot=True)

In [ ]:
# for i_fold, (th, proba) in enumerate(zip(ths, probas)):
#     print(u.f1_flexible(proba, th_start=0, th_stop=1, steps=20, gts=val.y)[0])
#     print(f1_score(y_pred=proba > th, y_true=val.y))
#     print(sum(proba > th))
#     print()

In [ ]:
# y_pred_vote = m.vote_predict(probas, ths)
# f1_score(y_pred=y_pred_vote, y_true=val.y)

In [ ]:
# models = m.load_models('/home/AlekseySh/code/comp/results/cb/2020-02-02 02:03:13.874913/')

In [ ]:
# pred = predict_multi_models(models, ths, val_pool)

In [ ]:
# u.estimate(models[1], val_pool, val_pool.get_label())

# CV

In [ ]:
train_total = u.select_by_time(train_full, '2016-07-01', '2019-01-01', 'datetime')
pool = Pool(data=train_total[all_cols], label=train_total['y'], cat_features=cat_cols)

In [ ]:
test_pool = Pool(data=test[all_cols], label=test['y'], cat_features=cat_cols)

In [ ]:
params = {
    'learning_rate': 0.1,
    'scale_pos_weight': 5,
    'has_time': True,
    'one_hot_max_size': 1000,
    'depth': 5,
    'l2_leaf_reg': 6,
    'grow_policy': 'SymmetricTree', # SymmetricTree, Lossguide, Depthwise
    
    'loss_function': 'Logloss',
    'task_type': 'CPU',
    # 'eval_metric': 'F1', # m.FlexibleF1(0, 1, 20)
    'custom_loss': 'F1'
}

In [ ]:
cv_res = cv(
   pool=pool, 
   params=params, 
   iterations=600,
   fold_count=5, 
   seed=42, 
   shuffle=True, 
   stratified=True,
   as_pandas=True,
   verbose=1,
   plot=True,
   early_stopping_rounds=600,
   type='Classical')

In [ ]:
params = {
    'iterations': 150,
    'learning_rate': 0.1,
    'scale_pos_weight': 5,
    'has_time': True,
    'one_hot_max_size': 1000,
    'depth': 5,
    'l2_leaf_reg': 6,
    'grow_policy': 'SymmetricTree', # SymmetricTree, Lossguide, Depthwise
    
    'loss_function': 'Logloss',
    'task_type': 'CPU',
    'custom_loss': 'F1',
    'train_dir': 'train_on_full'
}

cls = CatBoostClassifier(**params)

cls.fit(
    pool,
    verbose=10,
    plot=True
)

In [ ]:
proba_test = cls.predict_proba(test_pool)[:, 1]
proba_train = cls.predict_proba(pool)[:, 1]

In [ ]:
u.estimate(cls, pool, pool.get_label())

In [ ]:
cls.save_model('../results/cb_150_it_on_full.pt')

In [ ]:
w_unk = train_total.traffic_unknown == '1'
u.f1_flexible(proba_train[w_unk], train_total.y[w_unk], 0, 1, 20)
u.f1_flexible(proba_train[~w_unk], train_total.y[~w_unk], 0, 1, 20)

# 1 FOLD VAL

In [ ]:
# field_combs = [
#     tuple([('segment_id')]), tuple(['hour']), tuple(['weekday']), tuple(['month']),
#     ('segment_id', 'hour'), ('segment_id', 'weekday'), ('segment_id', 'month'), ('weekday', 'month'), ('hour', 'weekday'),
#     ('segment_id', 'weekday', 'hour'), ('segment_id', 'weekday', 'month')
# ]

# ones = u.read_ones('../data/train.csv')
# ones['segment_id'] = ones['sid']
# u.add_more_time(ones)

# train = u.select_by_time(train_full, '2018-01-01', '2018-10-01', 'datetime')
# u.add_statistic(data=train, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2018-01-01',
#                )

# val = u.select_by_time(train_full, '2018-10-01', '2019-01-01', 'datetime')
# u.add_statistic(data=val, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2018-10-01',
#                )

# u.add_statistic(data=test, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2019-01-01',
#                )


# hist_cols = list(filter(lambda x: x.startswith('hist_'), train.columns.values))

# all_cols += hist_cols
# cont_cols += hist_cols


In [ ]:
train = pd.concat([u.select_by_time(train_full, '2017-07-01', '2018-01-01', 'datetime'),
                   u.select_by_time(train_full, '2018-04-01', '2019-01-01', 'datetime')])
train.reset_index(inplace=True)

val = u.select_by_time(train_full, '2018-01-01', '2018-04-01', 'datetime')


# train = u.select_by_time(train_full_high, '2016-10-01', '2018-10-01', 'datetime')
# val = u.select_by_time(train_full_high, '2018-10-01', '2019-01-01', 'datetime')

train_pool = Pool(data=train[all_cols], label=train['y'], cat_features=cat_cols)
val_pool =  Pool(data=val[all_cols], label=val['y'], cat_features=cat_cols)
test_pool = Pool(data=test[all_cols], cat_features=cat_cols)

In [ ]:
# --bootstrap-type
# --bagging-temperature
# --random-strength
# --depth
# --grow-policy
# --max-leaves
# --rsm
# --boost-from-average
# --model-shrink-rate

params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'scale_pos_weight': 5,
    'has_time': True,
    'one_hot_max_size': 1000,
    'depth': 5,
    'l2_leaf_reg': 6,
    'grow_policy': 'SymmetricTree', # SymmetricTree, Lossguide, Depthwise
    
    'loss_function': 'Logloss',
    'task_type': 'CPU',
    'use_best_model': True,
    'eval_metric': m.FlexibleF1(0, 1, 20)
}

In [ ]:
cls = CatBoostClassifier(**params)

cls.fit(
    train_pool,
    eval_set=val_pool,
    plot=True,
    verbose=1,
    early_stopping_rounds=40,
)

In [ ]:
# cls_grid = CatBoostClassifier(**{
#     'loss_function': 'Logloss',
#     'task_type': 'GPU',
#     'eval_metric': 'F1'
# })

# param_grid = {'learning_rate': [0.05, 0.1, 0.15, 0.2],
#               'depth': [2, 3, 4],
#               'one_hot_max_size': [100, 255],
#               'iterations': [1000],
#               'class_weights': [[1, 3], [1, 5], [1, 10], [1, 20]],
#               'early_stopping_rounds': [150],
#               'l2_leaf_reg': [6, 12, 20],
#               'grow_policy': ['SymmetricTree', 'Lossguide', 'Depthwise']
#              }

# grid_search_result = cls_grid.randomized_search(
#             param_grid,
#             X=train_pool,
#             cv=2,
#             partition_random_seed=42,
#             stratified=True,
#             train_size=0.8,
#             verbose=True,
#             plot=True,
#             n_iter=50)

In [ ]:
th = u.estimate(cls, val_pool, y_true=val_pool.get_label(), th_start=0, th_stop=1, steps=100)

In [ ]:
proba_val = cls.predict_proba(val_pool)[:, 1]
proba_test = cls.predict_proba(test_pool)[:, 1]

In [ ]:
pred_val = proba_val > th
print(sum(pred_val))

print(f1_score(y_true=val.y, y_pred=pred_val))

# heuristic
pred_val[val.hour.astype(int) <= 3] = 0
pred_val[val.hour.astype(int) >= 22] = 0

print(f1_score(y_true=val.y, y_pred=pred_val))

In [ ]:
# cls = CatBoostClassifier()
# cls.load_model('../results/model_cb.pt')

cls.save_model('../results/cb/model_1206.pt')

In [ ]:
# from pprint import pprint
# ii = np.argsort(-1 * cls.feature_importances_)[-50:]
# pprint(list(zip(np.array(cls.feature_importances_)[ii], np.array(cls.feature_names_)[ii])))

# Submit

In [ ]:
pred = proba_test > 0.285

n_pred = sum(pred)
print(n_pred)

In [ ]:
submit = pd.DataFrame(
    data={'datetime x segment_id': test['datetime x segment_id'].values,
          'prediction': pred.astype(int)}
)
submit.to_csv(f'../results/submit_cb_{n_pred}.csv', index=False)